# Instructions:
1. Create a folder on your computer and put this jupyter notebook file in that folder
2. Go to https://exoplanets.nasa.gov/discovery/exoplanet-catalog/ and filter the search results by Planet Type to "Terrestrial" and by Discovery method to "Transit"
3. Choose a planet you want to view the light curve for
4. In the cell below this cell, enter the planet's name into the quotations
5. In jupyter notebook, go to the kernel tab, and press "Restart and Run All"
6. View the light curves you've generated in the folder named 'plots', which should be in the folder this jupyter notebook file is stored in
7. Have fun!

In [16]:
planet = "EPIC 201497682 b"

In [3]:
import matplotlib.pyplot as plt
import lightkurve as lk
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os

In [17]:
try: 
    print(planet)
    downloadNum = 0;
    try:
        os.makedirs("plots/" + planet)
    except:
        print('dir ' + ' already exists')
    baseWeb = "https://exoplanetarchive.ipac.caltech.edu/overview/"
    try:
        page = requests.get(baseWeb+planet).text
    except:
        raise Exception('could not access webpage of ' + planet)
    soup = BeautifulSoup(page, "lxml")
    planetInfo = soup.find_all("div", {"id":planet.replace(' ','-').replace('.','-')})
    allRows = planetInfo[0].find_all(class_="tooltip_wrapper")
    epochIndex = []
    periodIndex = []
    j = 0
    for item in allRows:
        if item['title'].__eq__('Time of Conjunction (Transit Midpoint)'):
            epochIndex.append(j)
        elif item['title'].__eq__('Orbital Period'):
            periodIndex.append(j)
        j = j + 1
    dataSoup = planetInfo[0].find_all('td', {'data-index': '0'})
    allDatas = []
    for item in dataSoup:
        allDatas.append(item.get_text())
    try:
        epochTime = float(allDatas[epochIndex[0]+1].split('±')[0].replace('\n',''));
    except:
        epochTime = float(allDatas[epochIndex[0]+1].split('+')[0].replace('\n',''));
    try:
        periodTime = float(allDatas[periodIndex[0]+1].split('±')[0].replace('\n',''));
    except:
        periodTime = float(allDatas[periodIndex[0]+1].split('+')[0].replace('\n',''));
    epochTime = epochTime - 2454833
    print([epochTime, periodTime])
    search_result = lk.search_lightcurve(planet)
    try:
        search_result[downloadNum]
        lcf = search_result[downloadNum].download()
    except:
        raise Exception('could not download data of ' + planet)
        
    plot = lcf.plot();
    fig = plot.get_figure();
    fig.savefig('plots/'+planet+'/base'+str(downloadNum)+'.png')

    ax = lcf.plot(label='PDCSAP Flux', normalize=True)
    plot = lcf.plot(column='sap_flux', normalize=True, label='SAP Flux', ax=ax);
    fig = plot.get_figure();
    fig.savefig('plots/'+planet+'/PDCSAP'+str(downloadNum)+'.png')

    pdcsap = lcf.flux
    sap =lcf.sap_flux
    finalData = lcf.remove_nans().flatten(window_length=401).fold(period=periodTime,  epoch_time=epochTime).bin(time_bin_size=0.025);

    plot = finalData.plot()
    fig = plot.get_figure();
    fig.savefig('plots/'+planet+'/Flattened'+str(downloadNum)+'.png')

    plt.close('all')
except:
    raise Exception(planet + ' plots ran into an error during generation')
    

EPIC 201497682 b
dir  already exists
[2017.0263000000268, 2.13174]
